# Testing Calibration V2Worksheet

## 1. Setup
This section load python objects and variables please check the ```cam``` variable

### 1.1 Load python objects
Loads ```scope``` and ```lamp``` python objects for reading sekonic and communication with the lamp

In [338]:
cam = "http://192.168.2.224/video_feed"
whiteTreshold = 150

%run calibration_setup_v2.ipynb

### 1.2 Connect to lamp
Lamp needs to be closed before it can be opened again.
If you dont know the serial port where the lamp is connected use the Find COM port section
1st You need to setup sekonic and feedback camera (the areas where the data will be read, 1st -green value is set up so it would be possibel to get dUV - sign location), then close the lamp and start again with the scope objects

In [58]:
lamp1.close()

In [60]:
lamp1 = apollo_device(ComPort='COM6')
lamp1.getId()
lamp1.macId = lamp1.macId.replace(":","")
lamp1.macId

'782184456C74'

In [61]:
lamp1.color = [200,50,50,500,0]
lamp1.update()

#### [Optional] Find COM port

In [9]:
# Get COM port list
import serial.tools.list_ports

com_ports = list(serial.tools.list_ports.comports())

for port, desc, hwid in com_ports:
    print(str(port)+" "+str(desc))

COM5 USB Serial Port (COM5)
COM6 USB Serial Port (COM6)


### 1.3 Set Sekonic camera areas
Set the area where to "read" the Tcp, dUV and lux variables
Once the object is initalized tune the paramters and test by ```update()```
To save the paramteres use the Setup below to generate the ```saved_string``` and copy and paste it into funtion paramter and save the workbook

In [329]:
Brightness = scope(saved_string="self.slider_h.value = (229, 388)\nself.slider_v.value = (390, 448)\nself.minus_comp.value = False\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.')]) \n")

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [63]:
Brightness.update()

4090

In [64]:
dUV = scope(saved_string="self.slider_h.value = (250, 443)\nself.slider_v.value = (304, 368)\nself.minus_comp.value = True\nself.offset_x.value = -80\nself.offset_y.value = 2\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('20.0', '0,0'), ('0.6', '0.0'), ('0.8', '0.0')]) \n")

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [66]:
dUV.update()

-0.0288

In [67]:
Kelvin = scope(saved_string="self.slider_h.value = (254, 412)\nself.slider_v.value = (228, 286)\nself.minus_comp.value = False\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('G', '6')]) \n")

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [25]:
Kelvin.update()

5129

#### Setup

In [26]:
Brightness.save_string()

"self.slider_h.value = (229, 388)\nself.slider_v.value = (390, 448)\nself.minus_comp.value = False\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.')]) \n"

In [27]:
dUV.save_string()

"self.slider_h.value = (250, 443)\nself.slider_v.value = (304, 368)\nself.minus_comp.value = True\nself.offset_x.value = -80\nself.offset_y.value = 2\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('20.0', '0,0'), ('0.6', '0.0'), ('0.8', '0.0')]) \n"

In [28]:
Kelvin.save_string()

"self.slider_h.value = (254, 412)\nself.slider_v.value = (228, 286)\nself.minus_comp.value = False\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('G', '6')]) \n"

# Test

In [339]:
lamp1.close()

In [340]:
lamp1 = apollo_device(ComPort='COM6',SekBrightnes=Brightness, SekDuv=dUV, SekKelvin=Kelvin)
lamp1.getId()
lamp1.macId = lamp1.macId.replace(":","")
lamp1.macId

'782184456C74'

In [341]:
%%time

KelvinTemp = 5 # 0-5 [2800 3200 4800 5600 7800 10000]
BrightnessV = 6 # 1-8 [min->max]

lamp1.AdjustLux(BrightnessV,KelvinTemp, calibration_points)
lamp1.AdjustDuv(BrightnessV,KelvinTemp, calibration_points)
lamp1.AdjustKelvin(BrightnessV,KelvinTemp, calibration_points)

print(f'Temperature: {lamp1.SekKelvin.update()}K\ndUv: {lamp1.SekDuv.update()}\nBrightness: {lamp1.SekBrightnes.update()}lx')

Adjusting lamp brightness to 5000lx in 10000K with [1, 2, 1, 4]/[8, 16, 10, 42] steps
current brightness: 143, ideal 5000, diff 4857 : [105, 214, 126, 523, 0]
current brightness: 5520, ideal 5000, diff -520 : [113, 230, 136, 565, 0]
current brightness: 5120, ideal 5000, diff -120 : [105, 214, 126, 523, 0]
current brightness: 5070, ideal 5000, diff -70 : [104, 212, 125, 519, 0]
current brightness: 5030, ideal 5000, diff -30 : [103, 210, 124, 515, 0]
current brightness: 5000, ideal 5000, diff 0 : [102, 208, 123, 511, 0]
current brightness: 5040, ideal 5000, diff -40 : [103, 210, 124, 515, 0]
Adjusting lamp dUV to +/- 0.003 in 10000K with [0, 1, 0, 0]/[0, 30, 0, 0] steps
current dUV: -0.0005, ideal 0.0000, diff -0.0005 : [102, 208, 123, 511, 0]
current dUV: -0.0004, ideal 0.0000, diff -0.0004 : [102, 209, 123, 511, 0]
current dUV: -0.0003, ideal 0.0000, diff -0.0003 : [102, 210, 123, 511, 0]
current dUV: -0.0002, ideal 0.0000, diff -0.0002 : [102, 211, 123, 511, 0]
current dUV: 0.0, ideal

In [333]:
%%time

KelvinTemp = 5 # 0-5 [2800 3200 4800 5600 7800 10000]
BrightnessV = 4 # 1-8 [min->max]

lamp1.AdjustLux(BrightnessV,KelvinTemp, calibration_points)
lamp1.AdjustDuv(BrightnessV,KelvinTemp, calibration_points)
lamp1.AdjustKelvin(BrightnessV,KelvinTemp, calibration_points)

print(f'Temperature: {lamp1.SekKelvin.update()}K\ndUv: {lamp1.SekDuv.update()}\nBrightness: {lamp1.SekBrightnes.update()}lx')

Adjusting lamp brightness to 1250lx in 10000K with [1, 2, 1, 4]/[8, 16, 10, 42] steps
current brightness: 1320, ideal 1250, diff -70 : [28, 52, 32, 129, 0]
current brightness: 1270, ideal 1250, diff -20 : [27, 50, 31, 125, 0]
current brightness: 1230, ideal 1250, diff 20 : [26, 48, 30, 121, 0]
current brightness: 1280, ideal 1250, diff -30 : [27, 50, 31, 125, 0]
current brightness: 1240, ideal 1250, diff 10 : [26, 48, 30, 121, 0]
current brightness: 1280, ideal 1250, diff -30 : [27, 50, 31, 125, 0]
current brightness: 1240, ideal 1250, diff 10 : [26, 48, 30, 121, 0]
Adjusting lamp dUV to +/- 0.003 in 10000K with [0, 1, 0, 0]/[0, 30, 0, 0] steps
current dUV: -0.0001, ideal 0.0000, diff -0.0001 : [26, 48, 30, 121, 0]
current dUV: 0.0001, ideal 0.0000, diff 0.0001 : [26, 49, 30, 121, 0]
current dUV: -0.0002, ideal 0.0000, diff -0.0002 : [26, 48, 30, 121, 0]
current dUV: 0.0001, ideal 0.0000, diff 0.0001 : [26, 49, 30, 121, 0]
current dUV: -0.0002, ideal 0.0000, diff -0.0002 : [26, 48, 30,

In [335]:
%%time

KelvinTemp = 5 # 0-5 [2800 3200 4800 5600 7800 10000]
BrightnessV = 1 # 1-8 [min->max]

lamp1.AdjustLux(BrightnessV,KelvinTemp, calibration_points)
lamp1.AdjustDuv(BrightnessV,KelvinTemp, calibration_points)
lamp1.AdjustKelvin(BrightnessV,KelvinTemp, calibration_points)

print(f'Temperature: {lamp1.SekKelvin.update()}K\ndUv: {lamp1.SekDuv.update()}\nBrightness: {lamp1.SekBrightnes.update()}lx')

Adjusting lamp brightness to 156lx in 10000K with [1, 2, 1, 5]/[9, 18, 10, 45] steps
current brightness: 149, ideal 156, diff 7 : [4, 4, 4, 16, 0]
current brightness: 199, ideal 156, diff -43 : [5, 6, 5, 21, 0]
current brightness: 149, ideal 156, diff 7 : [4, 4, 4, 16, 0]
current brightness: 199, ideal 156, diff -43 : [5, 6, 5, 21, 0]
current brightness: 149, ideal 156, diff 7 : [4, 4, 4, 16, 0]
current brightness: 199, ideal 156, diff -43 : [5, 6, 5, 21, 0]
current brightness: 149, ideal 156, diff 7 : [4, 4, 4, 16, 0]
Adjusting lamp dUV to +/- 0.003 in 10000K with [0, 1, 0, 0]/[0, 30, 0, 0] steps
current dUV: -0.0015, ideal 0.0000, diff -0.0015 : [4, 4, 4, 16, 0]
current dUV: 0.0033, ideal 0.0000, diff 0.0033 : [4, 5, 4, 16, 0]
current dUV: -0.0015, ideal 0.0000, diff -0.0015 : [4, 4, 4, 16, 0]
current dUV: 0.0034, ideal 0.0000, diff 0.0034 : [4, 5, 4, 16, 0]
current dUV: -0.0015, ideal 0.0000, diff -0.0015 : [4, 4, 4, 16, 0]
current dUV: 0.0033, ideal 0.0000, diff 0.0033 : [4, 5, 4, 

In [337]:
%%time

KelvinTemp = 5 # 0-5 [2800 3200 4800 5600 7800 10000]
for lx in range(7):
    lamp1.AdjustLux(8-lx,KelvinTemp, calibration_points)
    lamp1.AdjustDuv(8-lx,KelvinTemp, calibration_points)
    lamp1.AdjustKelvin(8-lx,KelvinTemp, calibration_points)
    
    if (7-lx)> 0:
        calibration_points[KelvinTemp][7-lx] = [round(x/2) for x in calibration_points[KelvinTemp][8-lx]]
    
    print(f'Temperature: {lamp1.SekKelvin.update()}K\ndUv: {lamp1.SekDuv.update()}\nBrightness: {lamp1.SekBrightnes.update()}lx')

Adjusting lamp brightness to 20000lx in 10000K with [1, 2, 1, 5]/[9, 18, 10, 45] steps
current brightness: 20100, ideal 20000, diff -100 : [384, 728, 415, 1879, 0]
current brightness: 20000, ideal 20000, diff 0 : [383, 726, 414, 1874, 0]
current brightness: 20100, ideal 20000, diff -100 : [384, 728, 415, 1879, 0]
current brightness: 20000, ideal 20000, diff 0 : [383, 726, 414, 1874, 0]
current brightness: 20100, ideal 20000, diff -100 : [384, 728, 415, 1879, 0]
current brightness: 20000, ideal 20000, diff 0 : [383, 726, 414, 1874, 0]
current brightness: 20100, ideal 20000, diff -100 : [384, 728, 415, 1879, 0]
Adjusting lamp dUV to +/- 0.003 in 10000K with [0, 1, 0, 0]/[0, 30, 0, 0] steps
current dUV: -0.0001, ideal 0.0000, diff -0.0001 : [383, 726, 414, 1874, 0]
current dUV: 0.0, ideal 0.0000, diff 0.0 : [383, 727, 414, 1874, 0]
current dUV: -0.0001, ideal 0.0000, diff -0.0001 : [383, 728, 414, 1874, 0]
current dUV: 0.0, ideal 0.0000, diff 0.0 : [383, 729, 414, 1874, 0]
current dUV: 0.

In [345]:
lamp1.color = [0, 0, 0, 0, 0]
lamp1.update()